# 🌊 Fit one shoreline

This notebook chooses one combination of
- subset `['all', 'exo', 'solar']` 
- type of atmosphere `['any', 'CO2']`
- whether to include uncertaintes `[True, False]` 
and fits a cosmic shoreline for that sample.

This should be run after `curate-and-label-planets.ipynb`, which is needed to create the organized dataset that will be loaded and fit here. 

This notebook can be run on its own, or as part of a loop in `fit-many-shorelines.ipynb`.


## 📓 Set up the fit.

To allow this notebook to be run as part of a loop from within another notebook, we use `try/except` statements below to check whether a variable has already been defined (in an enclosing notebook) before defining it ourselves here.

In [ ]:
from shoreline import * 

In [ ]:
# what subset and atmosphere kind should we fit? 
# (any choice of {[all, exo, solar]}-{any, CO2})
try:
    subset_and_kind
except NameError:
    subset_and_kind = 'all-CO2'

# should we include uncertainties? 
try:
    uncertainties
except NameError:
    uncertainties = True

# set up the sampling parameters
try:
    num_warmup
    num_samples
    num_chains
except NameError:
    num_warmup=5000
    num_samples=50000
    num_chains=4


print(f"""
🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️
🏝️ Let's fit a cosmic shoreline for '{subset_and_kind}' with uncertainties={uncertainties}. 🏝️
🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️🏝️
""")

If a posterior already exists, let's not run it again. To force it to run again, delete (or move) the posterior `.nc` file. (If being called from within another notebook, these `AssertionError`s will move on to the next fit that needs to be done).

In [ ]:
fileprefix = f"{subset_and_kind}-uncertainties={uncertainties}"
import os
if os.path.exists(f"posteriors/{fileprefix}-numpyro.nc"):
    print(f'''
    We're skipping a shoreline fit for {subset_and_kind} with uncertainties {uncertainties}
    because 'posteriors/{fileprefix}-numpyro.nc' exists.
    ''')
    assert False

## 💨 Assemble planet properties. 
Let's load the results of a notebook that used `exoatlas` to assemble some populations of planets and label them with atmospheres being present, absent, or unknown. Once it runs, we'll have access to `exoatlas` populations and `astropy` tables with all the data we need.

In [ ]:
A = load_organized_populations()

In [ ]:
list(A.keys())

In [ ]:
labeled = A[subset_and_kind]

In [ ]:
labeled

In [ ]:
data = convert_labeled_populations_into_table(labeled)

In [ ]:
data.show_in_notebook()

## 🦾 Define a shoreline model.

Let's write down the equation for our shoreline. 
- $\sf f$ is the average relative insolation the planet receives, $\sf f = L_\star/4\pi a^2$
- $\sf v_{esc}$ is the escape velocity at the planet's surface 
- $\sf L_\star$ is the star's bolometric luminosity


We construct a generative model that tries to explain the atmosphere labels $A_{\sf i}$ for planet $i$ using the predictors $f_{\sf i}$, $v_{\sf esc, i}$, $L_{\sf \star, i}$. We first define a cosmic shoreline flux $f_{\sf shoreline}$ for escape velocity $v_{\sf esc}$ and stellar luminosity $L_\star$ with the power law expression

$$\sf 
f_{\sf shoreline} = f_{\sf 0} \left(\frac{v_{\sf esc}}{v_{\sf esc, \oplus}}\right)^p\left(\frac{L_\star}{L_\odot}\right)^q
$$
where $f_{\sf 0}$, $p$, and $q$ are model parameters, $v_{\sf esc, \oplus} = 11.18$ km/s is Earth's escape velocity, and $L_\odot = 3.828 \times 10^{26}$ W is the Sun's luminosity. We compare all fluxes to Earth's average bolometric flux $f_\oplus = L_\odot/(4\pi a)^2 = 1361 \mathrm{W/m^2}$. This power law log transforms to a linear plane
$$\sf 
 \log_{10} \left(\frac{f_{\sf shoreline}}{f_\oplus}\right)  =  \log_{10} \left(\frac{f_{\sf 0}}{f_\oplus}\right)  + p \cdot  \log_{10} \left(\frac{v_{\sf esc}}{v_{\sf esc, \oplus}}\right) + q \cdot  \log_{10} \left(\frac{L_\star}{L_\odot}\right).
$$
We define a distance from this shoreline in log-flux as 
$$\sf
\Delta =  \log_{10} \left(\frac{f_{\sf }}{f_\oplus}\right) - \log_{10}\left(\frac{f_{\sf shoreline}}{f_\oplus}\right) = \log_{10}\left(\frac{f}{f_{\sf shoreline}}\right)
$$
which is similar to the Atmosphere Retention Metric from \citet{passRecedingCosmicShoreline2025}. We use this distance to describe the probability of each planet having an atmosphere with the logistic function as
$$\sf
p_{\sf i} = P(A_{\sf i} = 1 | \mathbf{x}_{\sf i}, \boldsymbol{\theta} ) = \frac{1}{1+e^{\Delta_{\sf i}/w}}
$$
where $\mathbf{x}_{\sf i} = [\log_{10}(f_{\sf i}/f_\oplus), \log_{10} (v_{\sf esc, i}/v_{\sf esc, \oplus}), \log_{10} (L_{\sf \star, i}/L_\odot)]$ are the predictors for each datum and $\boldsymbol{\theta} = [f_{\sf 0}, p, q, w]$ are the model parameters. This logistic function smoothly transitions from 1 when $f$ is below the shoreline to 0 above, with the width parameter $w$ describing the fuzziness of the shoreline, how quickly in $\log_{10} (f/f_\oplus)$ planets change from mostly having atmospheres to mostly not. 

In [ ]:
def log_f_shoreline(log_f_0=0.0, p=0.0, q=0.0, log_v=0, log_L=0):
    return log_f_0 + p * log_v + q * log_L

In [ ]:
def probability_of_atmosphere(
    log_f_0=1.0, p=4.0, q=0.0, ln_w=0, log_v=0, log_L=0, log_f=0
):
    distance_from_shoreline = log_f - log_f_shoreline(
        log_f_0=log_f_0, p=p, q=q, log_v=log_v, log_L=log_L
    )
    width_of_shoreline = jnp.exp(ln_w)
    return 1 / (1 + jnp.exp(distance_from_shoreline / width_of_shoreline))

## 🧹 Simplify data.

Let's transform the data into a simplified table that we can feed into our model, with all quantities (and their uncertainties) already log transformed.

In [ ]:

data_to_fit = data[['name', 'has_atmosphere']]
minimum_uncertainty = 0.01/jnp.log(10)

for k in ['relative_escape_velocity', 'relative_insolation', 'stellar_luminosity']:
    data_to_fit[f'log_{k}'] = jnp.log10(data[k])
    data_to_fit[f'sigma_log_{k}'] = data[f"{k}_uncertainty"]/data[k]/jnp.log(10)
    uncertainty_is_zero = data_to_fit[f'sigma_log_{k}'] == 0
    uncertainty_is_nonzero = data_to_fit[f'sigma_log_{k}'] > 0
    uncertainty_is_nonfinite = np.isfinite(data_to_fit[f'sigma_log_{k}']) == False
    print(f'''
    For {k}:
        {sum(uncertainty_is_zero)} with 0 uncertainty
        {sum(uncertainty_is_nonzero)} with >0 uncertainty
        {sum(uncertainty_is_nonfinite)} with non-finite uncertainty
    ''')
    data_to_fit[f'sigma_log_{k}'] = jnp.maximum( data_to_fit[f'sigma_log_{k}'], minimum_uncertainty)
data_to_fit

## ☘️ Fit with `numpyro`. 
Let's define our probabilistic model, with parameters drawn from priors and predictor measurements drawn from their 3D uncertainties. We'll compare predicted probabilities to actual atmosphere labels to define the likelihood.

In [ ]:
def model(data, uncertainties=True):


    # set up the four main parameters
    log_f_0 = numpyro.sample(
        "log_f_0", numpyro.distributions.Uniform(-50, 50)
    )
    p = numpyro.sample("p", numpyro.distributions.Uniform(-50, 50))
    q = numpyro.sample("q", numpyro.distributions.Uniform(-50, 50))
    ln_w = numpyro.sample(
        "ln_w",
        numpyro.distributions.Uniform(-6, 2),
    ) # w = 0.05 to 20
    w = numpyro.deterministic('w', jnp.exp(ln_w))

    # apply non-informative priors to slopes 
    numpyro.factor("log_prior_for_p", -1.5 * jnp.log(1 + p * p))
    numpyro.factor("log_prior_for_q", -1.5 * jnp.log(1 + q * q))

    # extract the labels from the data
    has_atmosphere = jnp.array(data["has_atmosphere"]).astype(float)

    if uncertainties:
        log_v = numpyro.sample('log_v', numpyro.distributions.Normal(data['log_relative_escape_velocity'], data['sigma_log_relative_escape_velocity']))
        log_f = numpyro.sample('log_f', numpyro.distributions.Normal(data['log_relative_insolation'], data['sigma_log_relative_insolation']))
        log_L = numpyro.sample('log_L', numpyro.distributions.Normal(data['log_stellar_luminosity'], data['sigma_log_stellar_luminosity']))
    else:
        log_v = data['log_relative_escape_velocity']
        log_f = data['log_relative_insolation']
        log_L = data['log_stellar_luminosity']
   

    # make probability predictions based on the model
    predicted_probability = probability_of_atmosphere(
        log_f_0=log_f_0,
        p=p,
        q=q,
        ln_w=ln_w,
        log_v=log_v,
        log_f=log_f,
        log_L=log_L,
    )

    # make sure predicted probabilities don't go nan
    safe_predicted_probability = jnp.where(
        jnp.isnan(predicted_probability), 0, predicted_probability # -jnp.inf, predicted_probability
    )
    numpyro.sample(
        "has_atmosphere",
        numpyro.distributions.Bernoulli(probs=safe_predicted_probability),
        obs=has_atmosphere,
    )



Let's sample from the posterior using the No U-Turns Sampler, which is magically good at exploring very high-dimensional probability distributions like ours.

In [ ]:
kernel = numpyro.infer.NUTS(model)
sampler = numpyro.infer.MCMC(
    kernel,
    num_warmup=num_warmup,
    num_samples=num_samples,
    num_chains=num_chains,
    progress_bar=True,
)

try:
    inference = az.from_netcdf("posteriors/{fileprefix}-numpyro.nc")
    print('not rerunning, because ')
except FileNotFoundError:


    key = jax.random.key(11)
    key, this_key = jax.random.split(key)
    sampler.run(this_key, data=data_to_fit, uncertainties=uncertainties)

## 🎨 See the results. 

Let's turn the posterior samples from `numpyro` into an `InferenceData` object from `arviz`, which we can use to summarize the results in lots of useful ways.

In [ ]:
inference = az.from_numpyro(sampler)
inference

Let's save the posterior samples (excluding the sampled data points, because they take up a lot of space).

In [ ]:
for k in ['log_L', 'log_f', 'log_v']:
    try:
        del inference.posterior[k]
    except KeyError:
        pass

mkdir("posteriors")
inference.to_netcdf(f"posteriors/{fileprefix}-numpyro.nc", groups="posterior")

Let's plot the traces from the independent chains, to make sure they agree and seem reasonable.

In [ ]:
var_names = ["log_f_0", "p", "q", "ln_w"]
az.plot_trace(
    inference,
    var_names=var_names,
    backend_kwargs={"constrained_layout": True},
)
plt.suptitle(subset_and_kind)
plt.savefig(f"posteriors/trace-{fileprefix}.pdf")


Let's save a summary of the parameters, with statistics and confidence intervals estimated from samples.

In [ ]:
import numpy as np
from astropy.stats import mad_std
mad_std



func_dict = {
    "mean": np.mean, 
    "std": np.std,
    "median": np.median,
    "mad_std": mad_std, 
    "lower": lambda x: np.median(x) - np.percentile(x, 50-68.3/2), 
    "upper": lambda x: np.percentile(x, 50+68.3/2) - np.median(x) }

s = az.summary(inference, var_names=var_names, stat_funcs=func_dict)
s.to_csv(f"posteriors/summary-{fileprefix}.csv")
s

Let's store a covariance matrix for the parameters (even though many samples might not be well-described by multivariate normal distributions).

In [ ]:
covariance_matrix = inference.to_dataframe(groups='posterior')[var_names].cov()
covariance_matrix.to_csv(f"posteriors/covariance-matrix-{fileprefix}.csv")

Let's make a rough corner plot to visualize the posterior.

In [ ]:
fig = plt.figure(figsize=(12, 12))
corner.corner(
    inference,
    var_names=var_names,
    fig=fig,
    color="black",
    hist_kwargs=dict(color="black", density=True),
    plot_density=False, plot_datapoints=False, show_titles=True)
plt.suptitle(subset_and_kind)
plt.savefig(f"posteriors/corner-{fileprefix}.pdf")